Collecting data of international borders from Wikipedia

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import codecs

In [2]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_countries_and_territories_by_land_and_maritime_borders"

Getting content of table with list of actual international borders

In [3]:
website_url = requests.get(WIKI_URL).text
soup = BeautifulSoup(website_url, 'lxml')
data_table = soup.find('table', {'class':'wikitable sortable'})
rows = data_table.findAll('tr')[3:]

In [4]:
countries = []
for i in rows:
    row = []
    country = i.td.a.string
    if country == None:
        continue
    row.append(country)
    if i.findAll("td")[-1].small != None:
        i.findAll("td")[-1].small.decompose()
    for j in i.findAll("td")[-1].findAll("a"):
        neig = j.string
        if neig != None and neig[0] != '[':
            row.append(neig)
    countries.append(row)

Saving data as CSV file

In [5]:
with codecs.open("../data/borders.csv", "w", "utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerows(countries)

Downloading COVID-19 data

In [3]:
COVID_URL = "https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"

In [4]:
r = requests.get(COVID_URL, allow_redirects=True)
open("../data/covid_data.csv", "wb").write(r.content)

913907

Loading data

In [2]:
import pandas as pd
import datetime

In [45]:
cases = pd.read_csv("../data/covid_data.csv")
cases['countriesAndTerritories'] = cases['countriesAndTerritories'].map(str.upper)

In [46]:
cases = cases.set_index(['countriesAndTerritories', 'dateRep'])

In [47]:
cases.index

MultiIndex([('AFGHANISTAN', '08/05/2020'),
            ('AFGHANISTAN', '07/05/2020'),
            ('AFGHANISTAN', '06/05/2020'),
            ('AFGHANISTAN', '05/05/2020'),
            ('AFGHANISTAN', '04/05/2020'),
            ('AFGHANISTAN', '03/05/2020'),
            ('AFGHANISTAN', '02/05/2020'),
            ('AFGHANISTAN', '01/05/2020'),
            ('AFGHANISTAN', '30/04/2020'),
            ('AFGHANISTAN', '29/04/2020'),
            ...
            (   'ZIMBABWE', '30/03/2020'),
            (   'ZIMBABWE', '29/03/2020'),
            (   'ZIMBABWE', '28/03/2020'),
            (   'ZIMBABWE', '27/03/2020'),
            (   'ZIMBABWE', '26/03/2020'),
            (   'ZIMBABWE', '25/03/2020'),
            (   'ZIMBABWE', '24/03/2020'),
            (   'ZIMBABWE', '23/03/2020'),
            (   'ZIMBABWE', '22/03/2020'),
            (   'ZIMBABWE', '21/03/2020')],
           names=['countriesAndTerritories', 'dateRep'], length=15698)

In [48]:
wczoraj = datetime.date(2020, 5, 8)

In [49]:
wczoraj.strftime('%d/%m/%Y')

'08/05/2020'

In [50]:
cases.loc['POLAND', wczoraj.strftime('%d/%m/%Y')]

day                               8
month                             5
year                           2020
cases                           307
deaths                           22
geoId                            PL
countryterritoryCode            POL
popData2018             3.79785e+07
continentExp                 Europe
Name: (POLAND, 08/05/2020), dtype: object

In [19]:
type(cases)

pandas.core.frame.DataFrame

In [30]:
with open("../data/borders.csv", mode="r", encoding="utf-8-sig") as file:
    borders = {}
    for line in file:
        neighbours = line.replace('\n','').replace(' ', '_').upper().split(',')
        borders[neighbours[0]] = neighbours[1:]
    file.close()

Creating mapping between names of countries in COVID dataset and borders dataset

In [48]:
for country in countries_from_covid_dataset:
    if country.upper() not in borders.keys():
        print(country)

Bonaire, Saint Eustatius and Saba
Brunei_Darussalam
Cases_on_an_international_conveyance_Japan
Congo
Cote_dIvoire
Czechia
Eswatini
Falkland_Islands_(Malvinas)
Gambia
Guinea_Bissau
Holy_See
Sao_Tome_and_Principe
Timor_Leste
United_Republic_of_Tanzania
United_States_of_America


In [56]:
mapping_countries_names = {
    'Bonaire, Saint Eustatius and Saba' : 'Netherlands',
    'Brunei_Darussalam' : 'Brunei',
    #'Cases_on_an_international_conveyance_Japan'  this is Diamond Princess,
    'Congo' : 'Republic_of_the_Congo',
    'Cote_dIvoire' : "Côte_d'Ivoire",
    'Czechia' : 'Czech_Republic',
    'Eswatini' : 'Eswatini_(Swaziland)' ,
    'Falkland_Islands_(Malvinas)' : 'Falkland_Islands',
    'Gambia' : 'The_Gambia',
    'Guinea_Bissau' : 'Guinea-Bissau',
    'Holy_See' : 'Vatican_City',
    'Sao_Tome_and_Principe' : 'São_Tomé_and_Príncipe' ,
    'Timor_Leste' : 'East_Timor',
    'United_Republic_of_Tanzania' : 'Tanzania' ,
    'United_States_of_America' : 'United_States',
}

In [57]:
for country in countries_from_covid_dataset:
    if country == 'Cases_on_an_international_conveyance_Japan':
        continue
    if country.upper() not in borders.keys():
        country = mapping_countries_names[country]
        if country.upper() not in borders.keys():
            print(country)

Testing

In [4]:
import Graph
import datetime

In [2]:
graph = Graph.get_graph()

In [3]:
graph.get_neighbours('Poland')

['BELARUS',
 'CZECH_REPUBLIC',
 'DENMARK',
 'GERMANY',
 'LITHUANIA',
 'RUSSIA',
 'SLOVAKIA',
 'SWEDEN',
 'UKRAINE']

In [7]:
yesterday = datetime.date.today() - datetime.timedelta(days=1)

In [8]:
yesterday

datetime.date(2020, 5, 8)

In [9]:
graph.get_stats('Poland', yesterday)

day                               8
month                             5
year                           2020
cases                           307
deaths                           22
geoId                            PL
countryterritoryCode            POL
popData2018             3.79785e+07
continentExp                 Europe
Name: (POLAND, 08/05/2020), dtype: object